In [1]:
from pwn import remote
from sage.all import GF, EllipticCurve, factor, discrete_log, crt

def disc_log(P, aP):
    dlog_list, fact_list = [], []
    order = P.order()
    facts = factor(order)

    print(f"[+] DISC LOG - Factor list: {facts}")
    for p, e in facts:
        print(f"\t[+] Seeing {p, e}")
        fact_list.append(p ** e)
        t = order // (p ** e)
        dlog_list.append(discrete_log(t * aP, t * P, operation='+'))

    print(f"\t[+] Chinese remainder theorem...")
    return crt(dlog_list, fact_list)

In [2]:
q = 2**128 - 159
a = 1
b = 1494

F = GF(q)
Ftwist = GF(q**2)

E = EllipticCurve(F, [a, b])
Etwist = EllipticCurve(Ftwist, [a, b])

print(f"[+] Order of E: {factor(E.order())}")
print(f"[+] Order of Etwist: {factor(Etwist.order())}")

[+] Order of E: 340282366920938463465004184633952524077
[+] Order of Etwist: 3 * 79 * 644899 * 1283505703 * 19385376821 * 89480282251 * 340282366920938463465004184633952524077


In [3]:
P = Etwist.lift_x(F(1))
print(f"[+] Order of P (with Px = 1) in Etwist: {factor(P.order())}")

[+] Order of P (with Px = 1) in Etwist: 3 * 79 * 644899 * 1283505703 * 19385376821 * 89480282251


In [5]:
r = remote("archive.cryptohack.org", 11718)

print(f"[+] P: {P}")
r.sendlineafter(b'x-coordinate: ', b'1')

aP = Etwist.lift_x(Ftwist(int(r.recvline())))
print(f"[+] aP: {aP}")
r.close()

# Solve
a = int(disc_log(P, aP))
print(f"[+] a: {a}")

print('[+] Flag: ECSC{' + bytes.fromhex(hex(a)[2:]).decode() + '}')

[x] Opening connection to archive.cryptohack.org on port 11718
[x] Opening connection to archive.cryptohack.org on port 11718: Trying 178.62.74.206
[ERROR] Could not connect to archive.cryptohack.org on port 11718
[-] Opening connection to archive.cryptohack.org on port 11718: Failed


PwnlibException: Could not connect to archive.cryptohack.org on port 11718